# Topics

- Collision detection
- 2D 
- 3D
- Z3 

In [29]:
# !pip install z3-solver

                                              0.0/58.9 MB ? eta -:--:--
                                              0.0/58.9 MB ? eta -:--:--
                                             0.0/58.9 MB 217.9 kB/s eta 0:04:31
                                             0.1/58.9 MB 435.7 kB/s eta 0:02:16
                                              0.4/58.9 MB 1.9 MB/s eta 0:00:32
                                              0.7/58.9 MB 3.0 MB/s eta 0:00:20
                                              1.0/58.9 MB 3.7 MB/s eta 0:00:16
                                              1.4/58.9 MB 4.2 MB/s eta 0:00:14
     -                                        1.7/58.9 MB 4.6 MB/s eta 0:00:13
     -                                        2.1/58.9 MB 4.9 MB/s eta 0:00:12
     -                                        2.4/58.9 MB 5.1 MB/s eta 0:00:12
     -                                        2.7/58.9 MB 5.3 MB/s eta 0:00:11
     --                                       3.1/58.9 MB 5.4 MB

In [30]:
import numpy as np
import pandas as pd
import math
import re
import sys
from shapely.geometry import Polygon
from matplotlib import pyplot as plt
from collections import Counter, OrderedDict, namedtuple, defaultdict, ChainMap
from queue import Queue
from copy import deepcopy
import networkx as nx
from functools import cmp_to_key
from itertools import product, permutations, combinations, combinations_with_replacement
from itertools import repeat
from functools import cache
from z3 import *
import json
import time

In [2]:
sys.setrecursionlimit(1500)

In [8]:
with open("24-input", "r") as file:
    lines = file.readlines()
data_raw = [line.replace("\n", "") for line in lines]
data_raw = "\n".join(data_raw)
data_raw

'237822270988608, 164539183264530, 381578606559948 @ 115, 346, -342\n287838354624648, 284335343503076, 181128681512377 @ -5, -84, 175\n341046208911993, 120694764237967, 376069872241870 @ -74, 129, -78\n275834119712623, 395388307575057, 177270820376760 @ 90, -111, -10\n284284433233698, 358506322947508, 169341917878543 @ 20, 133, 71\n236676388618224, 139432657314826, 343396041364471 @ 47, 28, 19\n271686440086412, 267306686527596, 183118696313003 @ 53, 273, 93\n286338526846979, 357920256353161, 178006715671210 @ 8, 227, -38\n277084490601628, 287132189587528, 259351816392501 @ 15, -16, 11\n271571599499731, 376770270435084, 104803601292067 @ 102, -12, 505\n318770864097708, 243720191032512, 285990120629205 @ -143, 505, -396\n295264015466268, 171618421304212, 325317744985951 @ -15, 75, -21\n295209980801646, 191803613346922, 309943723274129 @ -14, 20, 20\n198214675546904, 272335936655574, 258298762033323 @ 111, -68, 75\n288187702040868, 388917296420926, 171504207459427 @ -10, 94, -111\n1842739

In [5]:
test_data_raw = r"""19, 13, 30 @ -2,  1, -2
18, 19, 22 @ -1, -1, -2
20, 25, 34 @ -2, -2, -4
12, 31, 28 @ -1, -2, -1
20, 19, 15 @  1, -5, -3"""




Hailstone = namedtuple("Hailstone", ["px", "py", "pz", "vx", "vy", "vz"])
Point = namedtuple("Point", ["x", "y", "z"])

def preprocess_data (data):
    # dtype='U10'
    def parse_row(row):
        p = row.split(" @ ")[0].split(", ")
        v = row.split(" @ ")[1].split(", ")
        return int(p[0]), int(p[1]), int(p[2]), int(v[0]), int(v[1]), int(v[2])

    rows = [ Hailstone(*parse_row(row)) for row in data.split("\n")]
    return rows

test_data = preprocess_data(test_data_raw)
display(test_data)

[Hailstone(px=19, py=13, pz=30, vx=-2, vy=1, vz=-2),
 Hailstone(px=18, py=19, pz=22, vx=-1, vy=-1, vz=-2),
 Hailstone(px=20, py=25, pz=34, vx=-2, vy=-2, vz=-4),
 Hailstone(px=12, py=31, pz=28, vx=-1, vy=-2, vz=-1),
 Hailstone(px=20, py=19, pz=15, vx=1, vy=-5, vz=-3)]

In [9]:
data = preprocess_data(data_raw)
data

[Hailstone(px=237822270988608, py=164539183264530, pz=381578606559948, vx=115, vy=346, vz=-342),
 Hailstone(px=287838354624648, py=284335343503076, pz=181128681512377, vx=-5, vy=-84, vz=175),
 Hailstone(px=341046208911993, py=120694764237967, pz=376069872241870, vx=-74, vy=129, vz=-78),
 Hailstone(px=275834119712623, py=395388307575057, pz=177270820376760, vx=90, vy=-111, vz=-10),
 Hailstone(px=284284433233698, py=358506322947508, pz=169341917878543, vx=20, vy=133, vz=71),
 Hailstone(px=236676388618224, py=139432657314826, pz=343396041364471, vx=47, vy=28, vz=19),
 Hailstone(px=271686440086412, py=267306686527596, pz=183118696313003, vx=53, vy=273, vz=93),
 Hailstone(px=286338526846979, py=357920256353161, pz=178006715671210, vx=8, vy=227, vz=-38),
 Hailstone(px=277084490601628, py=287132189587528, pz=259351816392501, vx=15, vy=-16, vz=11),
 Hailstone(px=271571599499731, py=376770270435084, pz=104803601292067, vx=102, vy=-12, vz=505),
 Hailstone(px=318770864097708, py=243720191032512, 

In [11]:
list(combinations(range(len(test_data)), 2))

[(0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (1, 2),
 (1, 3),
 (1, 4),
 (2, 3),
 (2, 4),
 (3, 4)]

In [17]:
A_test = Hailstone(1,1,0, 1,-1,0)
B_test = Hailstone(1,-1,0, 1, 1,0)

def calc_collision (A, B):
    if (B.vy / B.vx) == (A.vy / A.vx):
        return None

    x = (A.py - B.py - A.px * (A.vy / A.vx) + B.px * (B.vy / B.vx)) / ((B.vy / B.vx) - (A.vy / A.vx))
    y = A.py + (x - A.px) * (A.vy / A.vx)
    return x,y
calc_collision(A_test, B_test)

(2.0, 0.0)

In [27]:
def solution (data, val_min, val_max, verbose=False):


    collisions = 0
    for A, B in combinations(range(len(data)), 2):
        
        if point := calc_collision(data[A],data[B]):
            At = (point[0] - data[A].px) / data[A].vx
            Bt = (point[0] - data[B].px) / data[B].vx
            if (val_min <= point[0] <= val_max) and (val_min <= point[1] <= val_max) and (At > 0) and (Bt > 0) :
                collisions += 1


    return collisions



sol = solution(data, 200000000000000, 400000000000000)
# sol = solution(test_data, 7, 27, verbose=True)

# print(dists)
print(sol)
# display(sum(sol))


19976


# Part 2

In [95]:
def solution (data, val_min, val_max, verbose=False):


    s = Solver()
    T = [Int(f"t{i}") for i in range(len(data))]
    x = Int("x")
    y = Int("y")
    z = Int("z")
    vx = Int("vx")
    vy = Int("vy")
    vz = Int("vz")
    for idx, hailstone in enumerate(data):
        s.add(T[idx] > 0)
        s.add(hailstone.px + T[idx] * hailstone.vx  == x + T[idx] * vx)
        s.add(hailstone.py + T[idx] * hailstone.vy  == y + T[idx] * vy)
        s.add(hailstone.pz + T[idx] * hailstone.vz  == z + T[idx] * vz)
        # s.add(x + T[idx] * vx >= val_min)
        # s.add(x + T[idx] * vx <= val_max)
        # s.add(y + T[idx] * vy >= val_min)
        # s.add(y + T[idx] * vy <= val_max)
        # s.add(z + T[idx] * vz >= val_min)
        # s.add(z + T[idx] * vz <= val_max)
        # s.add(hailstone.py + T[idx] * hailstone.vy  == test_data[idx + 1].py + T[idx+1] * test_data[idx + 1].vy)

    print(s.check())
    
    m = s.model()
    print(f"(x,y,z) = ({m[x]},{m[y]},{m[z]})")
    print(f"(vx,vy,vz) = ({m[vx]},{m[vy]},{m[vz]})")
    ts = ""
    for t in T:
        ts += f"{t} = {m[t]}, "
    print(ts)

    for idx, hailstone in enumerate(data):
        xt, yt, zt = m.eval(x + T[idx] * vx), m.eval(y + T[idx] * vy), m.eval(z + T[idx] * vz)
        print("Hailstone:", hailstone, " @ time", m[T[idx]], "and position", f"({xt}, {yt}, {zt})")

    return m.eval(x).as_long() + m.eval(y).as_long() + m.eval(z).as_long()



sol = solution(data, 200000000000000, 400000000000000)
# sol = solution(test_data, 7, 27, verbose=True)

# print(dists)
print(sol)
# display(sum(sol))


sat
(x,y,z) = (287838354624648,412952398656862,148587016955395)
(vx,vy,vz) = (-5,-250,217)
t0 = 416800696967, t1 = 774801537071, t2 = 771128323005, t3 = 126360367495, t4 = 142156855638, t5 = 983883961662, t6 = 278481285142, t7 = 115371367513, t8 = 537693201151, t9 = 152025748831, t10 = 224148619370, t11 = 742566084162, t12 = 819069575222, t13 = 772617923084, t14 = 69869483244, t15 = 488511229297, t16 = 969428804875, t17 = 618155411317, t18 = 809639670642, t19 = 381083049645, t20 = 878481670654, t21 = 339616404871, t22 = 167714314460, t23 = 209117366063, t24 = 922051928445, t25 = 418898598714, t26 = 948611581422, t27 = 536749752613, t28 = 329866313804, t29 = 589941509220, t30 = 397424300082, t31 = 474249500455, t32 = 292505943626, t33 = 822110128285, t34 = 610217085010, t35 = 691661340623, t36 = 561675368872, t37 = 70865008846, t38 = 980462834755, t39 = 655262359874, t40 = 517824175321, t41 = 906332507809, t42 = 193527901514, t43 = 120711461440, t44 = 311022308600, t45 = 358476902939, t